# Physics Informed Neural Networks <br> Airfoil Aerodymanics

## PINN

In [ ]:
import os
from dotenv import load_dotenv
import sys

In [ ]:
load_dotenv()

lib_dir = os.environ.get("LOCAL_LIB_DIR")
sys.path.append(lib_dir)

data_dir_airfoil = os.environ.get("LOCAL_DATA_DIR_AIRFOIL")
model_dir_airfoil = os.path.join(data_dir_airfoil, "models")

In [ ]:
import torch
import yaml
from airfoil_pinn import AirfoilPINN
from naca4digit_airfoil import Naca4DigitAirfoil
import utils

In [ ]:
device = utils.get_device()
# device = "cpu"
print(f"Device: {device}")

In [ ]:
with open("config_airfoil.yaml", 'r') as file:
    config = yaml.safe_load(file)

In [ ]:
# Air Density (rho)
# Air Dynamic viscosity (mu)
rho = config["rho"]
mu = config["mu"]

# m/s
in_velocity = config["in_velocity"]

# Domain limits
x_min = config["x_min"]
y_min = config["y_min"]
x_max = config["x_max"]
y_max = config["y_max"]

In [ ]:
# Define airfoil parameters (example: NACA 2412)
chord = 1.0
m = 0.02  # maximum camber
p = 0.4   # position of maximum camber
t = 0.12  # maximum thickness
num_points = 100

airfoil = Naca4DigitAirfoil(chord, m, p, t)

In [ ]:
hidden_units = config["hidden_units"]

pinn = AirfoilPINN(hidden_units, airfoil).to(device)

In [ ]:
optimizer = torch.optim.LBFGS(pinn.parameters(), lr=1, line_search_fn="strong_wolfe")
# optimizer = torch.optim.Adam(pinn.parameters(), lr=5e-4)
# optimizer = torch.optim.Adam(pinn.parameters(), lr=1e-6)
# optimizer = torch.optim.Adam(pinn.parameters())

In [ ]:
epochs = config["epochs"]
checkpoint_epochs = config["checkpoint_epochs"]

Nf = config["Nf"]   # num of collocation points -> pde evaluation
Nb = config["Nb"]   # num of points to evaluate boundary conditions
Ns = config["Ns"]   # num of points of the surface of the airfoil to evaluate boundary conditions
Nin = config["Nin"] # num of points inside the airfoil to evaluate boundary conditions

In [ ]:
pinn.train_pinn(
  epochs=epochs, 
  optimizer=optimizer, 
  Nf=Nf, 
  Nb=Nb, 
  Ns=Ns, 
  Nin=Nin,
  x_min=x_min, 
  y_min=y_min, 
  x_max=x_max, 
  y_max=y_max, 
  in_velocity=in_velocity, 
  mu=mu, 
  rho=rho, 
  device=device, 
  checkpoint_epochs=checkpoint_epochs, 
  model_dir=model_dir_airfoil, 
)

In [ ]:
pinn.plot_learning_curves(output_dir=model_dir_airfoil)